In [13]:
import numpy as np
import pandas as pd
data = pd.read_csv('ibm-hr-analytics-attrition.csv')
data.tail()


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8
1469,34,No,Travel_Rarely,628,Research & Development,8,3,Medical,1,2068,...,1,80,0,6,3,4,4,3,1,2


In [120]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import  StringIndexer, VectorAssembler
from pyspark.ml.feature import FeatureHasher

from pyspark.ml import Pipeline
from pyspark.sql.types import StructType, StructField, StringType,IntegerType

spark = SparkSession.builder.appName("Predict").getOrCreate()

In [133]:
schema = StructType([
 StructField("Age",IntegerType(),True),
 StructField("Attrition",StringType(),True),
 StructField("BusinessTravel",StringType(),True),
 StructField("DailyRate",IntegerType(),True),
 StructField("Department",StringType(),True),
 StructField("DistanceFromHome",IntegerType(),True),
 StructField("Education",IntegerType(),True),
 StructField("EducationField",StringType(),True),
 StructField("EmployeeCount",IntegerType(),True),
 StructField("EmployeeNumber",IntegerType(),True),
 StructField("EnvironmentSatisfaction",IntegerType(),True),
 StructField("Gender",StringType(),True),
 StructField("HourlyRate",IntegerType(),True),
 StructField("JobInvolvement",IntegerType(),True),
 StructField("JobLevel",IntegerType(),True),
 StructField("JobRole",StringType(),True),
 StructField("JobSatisfaction",IntegerType(),True),
 StructField("MaritalStatus",StringType(),True),
 StructField("MonthlyIncome",IntegerType(),True),
 StructField("MonthlyRate",IntegerType(),True),
 StructField("NumCompaniesWorked",IntegerType(),True),
 StructField("Over18",StringType(),True),
 StructField("OverTime",StringType(),True),
 StructField("PercentSalaryHike",IntegerType(),True),
 StructField("PerformanceRating",IntegerType(),True),
 StructField("RelationshipSatisfaction",IntegerType(),True),
 StructField("StandardHours",IntegerType(),True),
 StructField("StockOptionLevel",IntegerType(),True),
 StructField("TotalWorkingYears",IntegerType(),True),
 StructField("TrainingTimesLastYear",IntegerType(),True),
 StructField("WorkLifeBalance",IntegerType(),True),
 StructField("YearsAtCompany",IntegerType(),True),
 StructField("YearsInCurrentRole",IntegerType(),True),
 StructField("YearsSinceLastPromotion",IntegerType(),True),
 StructField("YearsWithCurrManager",IntegerType(),True)
])

In [134]:
train_df = spark.read.csv('ibm-hr-analytics-attrition.csv', header=True, schema=schema)    

In [135]:
train_df.head(1)

[Row(Age=41, Attrition='Yes', BusinessTravel='Travel_Rarely', DailyRate=1102, Department='Sales', DistanceFromHome=1, Education=2, EducationField='Life Sciences', EmployeeCount=1, EmployeeNumber=1, EnvironmentSatisfaction=2, Gender='Female', HourlyRate=94, JobInvolvement=3, JobLevel=2, JobRole='Sales Executive', JobSatisfaction=4, MaritalStatus='Single', MonthlyIncome=5993, MonthlyRate=19479, NumCompaniesWorked=8, Over18='Y', OverTime='Yes', PercentSalaryHike=11, PerformanceRating=3, RelationshipSatisfaction=1, StandardHours=80, StockOptionLevel=0, TotalWorkingYears=8, TrainingTimesLastYear=0, WorkLifeBalance=1, YearsAtCompany=6, YearsInCurrentRole=4, YearsSinceLastPromotion=0, YearsWithCurrManager=5)]

In [136]:
train_Pandas  = train_df.toPandas()
train_Pandas.tail(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8
1469,34,No,Travel_Rarely,628,Research & Development,8,3,Medical,1,2068,...,1,80,0,6,3,4,4,3,1,2


In [143]:
#Attrition = train_df.select('Attrition').collect();
#train_df.printSchema()
columns = train_Pandas.keys().tolist()
#columns
train_Pandas.describe()
#train_Pandas["Education"].mean()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,...,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,802.485714,9.192517,2.912925,1.0,1024.865306,2.721769,65.891156,2.729932,2.063946,...,2.712245,80.0,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,403.509100,8.106864,1.024165,0.0,602.024335,1.093082,20.329428,0.711561,1.106940,...,1.081209,0.0,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,1.000000,30.000000,1.000000,1.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,465.000000,2.000000,2.000000,1.0,491.250000,2.000000,48.000000,2.000000,1.000000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,802.000000,7.000000,3.000000,1.0,1020.500000,3.000000,66.000000,3.000000,2.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,1157.000000,14.000000,4.000000,1.0,1555.750000,4.000000,83.750000,3.000000,3.000000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1499.000000,29.000000,5.000000,1.0,2068.000000,4.000000,100.000000,4.000000,5.000000,...,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000
